# Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd './drive/MyDrive/gaze_estimation'

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import random

from tqdm.auto import tqdm
from glob import glob
from scipy.io import loadmat


In [ ]:
def make_arbitrary_masking(N, ind) :
    indice = np.arange(0,N)
    mask = np.zeros(N, dtype=bool)
    mask[ind] = True
    return indice[~mask], indice[mask]

def k_fold_index(N = 150, k = 10, randomize = True, SEED = 10) :
    indice = np.arange(0,N)
    if randomize is True :
        np.random.seed(SEED)
        np.random.shuffle(indice)
    result = []
    for fold in np.split(indice, k) :
        result.append(make_arbitrary_masking(N, fold))
    return result

# Data Path

In [ ]:
id_vector = np.load('../mpii_dataset/full_ids.npy')
images = np.load('../mpii_dataset/full_images.npy')
head_poses = np.load('../mpii_dataset/full_2d_hps.npy')
gazes = np.load('../mpii_dataset/full_2d_gazes.npy')

In [ ]:
loocv_indices = [np.where(id_vector == ids)[0] for ids in np.unique(id_vector)]

loocv_ids = np.concatenate([id_vector[ind][np.newaxis,:] for ind in loocv_indices])
loocv_images = np.concatenate([images[ind][np.newaxis,:] for ind in loocv_indices])
loocv_hps = np.concatenate([head_poses[ind][np.newaxis,:] for ind in loocv_indices])
loocv_gazes = np.concatenate([gazes[ind][np.newaxis,:] for ind in loocv_indices])

In [ ]:
cv_indice_list = [k_fold_index(N=3000, k=10, randomize=True, SEED=100 + i) for i in range(15)]
fold_indice_list = [np.concatenate([cv_indice_list[i][fold][1][np.newaxis,:] for i in range(15)]) for fold in range(10)]

In [ ]:
within_ids = np.concatenate([np.concatenate([ids[fold_indice_list[fold][0]] for ids in loocv_ids])[np.newaxis,] for fold in range(10)])
within_images = np.concatenate([np.concatenate([images[fold_indice_list[fold][0]] for images in loocv_images])[np.newaxis,] for fold in range(10)])
within_hps = np.concatenate([np.concatenate([hps[fold_indice_list[fold][0]] for hps in loocv_hps])[np.newaxis,] for fold in range(10)])
within_gazes = np.concatenate([np.concatenate([gazes[fold_indice_list[fold][0]] for gazes in loocv_gazes])[np.newaxis,] for fold in range(10)])

In [ ]:
np.save('../mpii_dataset/loocv_ids', loocv_ids)
np.save('../mpii_dataset/loocv_images', loocv_images)
np.save('../mpii_dataset/loocv_2d_hps', loocv_hps)
np.save('../mpii_dataset/loocv_2d_gazes', loocv_gazes)

np.save('../mpii_dataset/within_ids', within_ids)
np.save('../mpii_dataset/within_images', within_images)
np.save('../mpii_dataset/within_2d_hps', within_hps)
np.save('../mpii_dataset/within_2d_gazes', within_gazes)